수집하는 데이터
1. close_price
2. open_price
3. high_price
4. low_price
5. volume 

__해당 코드는, 금일 데이터를 수집하는 코드이다. 매일 장 마감 후에 자동화로 수집을 진행예정__

In [1]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

import datetime
# 파일 입출력 담당
from file_rw import *

import os
# delete directory and files at once
import shutil

In [2]:

# 네이버 금융으로 url 변경
def crawl(c_code):
    url = "https://finance.naver.com/item/main.nhn?code="
    url += str(c_code)
    html = urlopen(url)

    soup =  BeautifulSoup(html, "html.parser")
    
    #0.종가
    close_price = soup.find_all("dd")[3]
    close_price = re.sub(pattern = ',' ,repl = '', string = str(close_price))
    close_price = re.findall(pattern = r'현재가 \d*', string = close_price)[0].split(' ')[1]

    #1.시가
    open_price = soup.find_all("dd")[5]
    open_price = re.sub(pattern = ',' ,repl = '', string = str(open_price))
    open_price = re.findall(pattern = r'시가 \d*', string = open_price)[0].split(' ')[1]

    #2.고가
    high_price = soup.find_all("dd")[6]
    high_price = re.sub(pattern = ',' ,repl = '', string = str(high_price))
    high_price = re.findall(pattern = r'고가 \d*', string = high_price)[0].split(' ')[1]

    #3.저가
    low_price = soup.find_all("dd")[8]
    low_price = re.sub(pattern = ',' ,repl = '', string = str(low_price))
    low_price = re.findall(pattern = r'저가 \d*', string = low_price)[0].split(' ')[1]

    #4.거래량
    volume = soup.find_all("dd")[10]
    volume = re.sub(pattern = ',' ,repl = '', string = str(volume))
    volume = re.findall(pattern = r'거래량 \d*', string = volume)[0].split(' ')[1]
    
    res = [close_price, open_price, high_price, low_price, volume]
    
    return res


jongmok_code = pd.read_csv("../kospi-stock-code.csv")


daily_data = {}

for i in range(len(jongmok_code)):
    c_name = jongmok_code.iloc[i]["회사명"]
    daily_data[c_name] = {}
    c_code = str(jongmok_code.iloc[i]["종목코드"])
    
    
    #종목코드는 무조건 6글자
    if len(c_code) < 6:
        ii = 6 - len(c_code)
        c_code = '0' * ii + str(c_code)
        
    res = crawl(c_code)
    
    close_price = res[0]
    open_price = res[1]
    high_price = res[2]
    low_price = res[3]
    volume = res[4]
    
    daily_data[c_name]["code"] = c_code
    daily_data[c_name]["close_price"] = close_price
    daily_data[c_name]["open_price"] = open_price
    daily_data[c_name]["high_price"] = high_price
    daily_data[c_name]["low_price"] = low_price
    daily_data[c_name]["volume"] = volume
    
    
    print(c_code, c_name, 'close : ', close_price, ', open :', open_price)
    
    #많은 양을 수집하는데 시간 오래걸려서, 일단 테스트용으로 5개종목 데이터만 수집
    if i == 5:
        break

        
#수집한 일별 데이터 적재
result = []
for k in daily_data:
    result.append(daily_data[k])

302440 SK바이오사이언스 close :  152000 , open : 150500
378850 화승알앤에이 close :  6390 , open : 6700
950210 프레스티지바이오파마 close :  33850 , open : 35150
248070 솔루엠 close :  25050 , open : 24750
375500 DL이앤씨 close :  133000 , open : 135000
365550 ESR켄달스퀘어리츠 close :  6460 , open : 6450


In [3]:
#날짜데이터로 저장
now = datetime.datetime.now()
nowDate = now.strftime('%Y_%m_%d')

#Now Date String
nds = str(nowDate)

#File Name
fn1 = str(nds) + '/dailyUpdateData.json'


"""ec2 서버 내에 json 파일로 저장시의 경로 설정

directory_ = ??? # 폴더명
dn = '/home/ec2-user/app/' + directory_ +'/data/' + str(nds)
if os.path.isdir(dn):
    shutil.rmtree(dn)
os.mkdir(dn)
"""

print(fn1)

2021_05_02/dailyUpdateData.json


In [9]:
#저장 형태는 이와 같지만, Json 형태로 저장 예정
print('데이터 수집 일 : ', nds)
pd.DataFrame(result)

데이터 수집 일 :  2021_05_02


,code,close_price,open_price,high_price,low_price,volume
0,302440,152000,150500,156000,144500,916879
1,378850,6390,6700,6860,6350,558523
2,950210,33850,35150,35500,33800,883896
3,248070,25050,24750,25300,24250,547104
4,375500,133000,135000,135500,131000,269461
5,365550,6460,6450,6500,6440,317140


In [11]:
#파일 네임은 fn1로, json 파일로 저장
print(result)

#JsonWrite(fn1, result)

[{'code': '302440', 'close_price': '152000', 'open_price': '150500', 'high_price': '156000', 'low_price': '144500', 'volume': '916879'}, {'code': '378850', 'close_price': '6390', 'open_price': '6700', 'high_price': '6860', 'low_price': '6350', 'volume': '558523'}, {'code': '950210', 'close_price': '33850', 'open_price': '35150', 'high_price': '35500', 'low_price': '33800', 'volume': '883896'}, {'code': '248070', 'close_price': '25050', 'open_price': '24750', 'high_price': '25300', 'low_price': '24250', 'volume': '547104'}, {'code': '375500', 'close_price': '133000', 'open_price': '135000', 'high_price': '135500', 'low_price': '131000', 'volume': '269461'}, {'code': '365550', 'close_price': '6460', 'open_price': '6450', 'high_price': '6500', 'low_price': '6440', 'volume': '317140'}]
